In [2]:
import pandas as pd
import numpy as np
import os

import tensorflow as tf
from tensorflow.keras import layers, losses, models, optimizers

import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from numpy.lib.stride_tricks import sliding_window_view
from sklearn.preprocessing import MinMaxScaler

import vae as V

def data_scaling(data):             #스케일러, main
    scaler = MinMaxScaler()
    data = scaler.fit_transform(data)

    return data


def data_sliding(data, n_features, Time_window): #데이터 밀기, main
    # reshape_num = data.shape[0] - Time_window + 1
    data = sliding_window_view(data, (Time_window, n_features))
    #   data = data.reshape(reshape_num, Time_window, n_features) <-필요없음

    return data


def data_load_processing(FILENAME, Time_window=None): #데이터 로딩 및 스케일러 사용, main
    df = pd.read_csv(FILENAME)
    df = df.drop(['Time'], axis=1)
    n_features = df.shape[1]
    x_train, x_test = train_test_split(df, test_size=0.9, shuffle=False)
    #x_validation, x_test = train_test_split(x_test, test_size=0.5, shuffle=False)

    x_train = data_scaling(x_train)
    x_test = data_scaling(x_test)
    #x_validation = data_scaling(x_validation)

    x_train = data_sliding(x_train, n_features, Time_window)
    x_test = data_sliding(x_test, n_features, Time_window)
    #x_validation = data_sliding(x_validation, n_features, Time_window)

    return x_train, x_test #, x_validation


def sampling(inputs):
    z_mean, z_log_var = inputs
    batch = tf.shape(z_mean)[0]
    dim = tf.shape(z_mean)[1]
    epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
    return z_mean + tf.exp(0.5 * z_log_var) * epsilon

class VAE(tf.keras.Model):
    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder
        self.total_loss_tracker = tf.keras.metrics.Mean(name="total_loss")
        self.reconstruction_loss_tracker = tf.keras.metrics.Mean(
            name="reconstruction_loss"
        )
        self.kl_loss_tracker = tf.keras.metrics.Mean(name="kl_loss")

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = self.encoder(data)
            reconstruction = self.decoder(z)
            reconstruction_loss = tf.reduce_mean(
                tf.reduce_sum(
                    tf.keras.losses.binary_crossentropy(data, reconstruction), axis=(1, 2)
                )
            )
            kl_loss = -0.5 * (1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var))
            kl_loss = tf.reduce_mean(tf.reduce_sum(kl_loss, axis=1))
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss)
        self.reconstruction_loss_tracker.update_state(reconstruction_loss)
        self.kl_loss_tracker.update_state(kl_loss)
        return {
            "loss": self.total_loss_tracker.result(),
            "reconstruction_loss": self.reconstruction_loss_tracker.result(),
            "kl_loss": self.kl_loss_tracker.result(),
        }


def vae_model(input_shape, latent_dim, kernel_size, strides, padding):
    inputs = tf.keras.Input(shape=input_shape)
    h = layers.Conv2D(32, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                      name="conv1", data_format="channels_first")(inputs)
    h = layers.Conv2D(64, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                      name="conv2", data_format="channels_first")(h)
    h = layers.Conv2D(128, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                      name="conv3", data_format="channels_first")(h)
    shape_save = h.get_shape().as_list()
    h = layers.Flatten()(h)
    h = layers.Dense(16, activation="relu")(h)

    # vae
    z_mean = layers.Dense(latent_dim, name='z_mean')(h)
    z_log_var = layers.Dense(latent_dim, name='z_log_var')(h)
    z = sampling([z_mean, z_log_var])
    encoder = tf.keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')

    # decoder
    latent_inputs = tf.keras.Input(shape=(latent_dim,))
    h = layers.Dense(shape_save[1] * shape_save[2] * shape_save[3], activation="relu")(latent_inputs)
    h = layers.Reshape((shape_save[1], shape_save[2], shape_save[3]))(h)
    h = layers.Conv2DTranspose(64, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                               data_format="channels_first")(h)
    h = layers.Conv2DTranspose(32, kernel_size=kernel_size, strides=strides, padding=padding, activation='relu',
                               data_format="channels_first")(h)
    decoder_outputs = layers.Conv2DTranspose(1,kernel_size=kernel_size, strides=strides, padding=padding, activation="relu",
                                             data_format="channels_first")(h)
    decoder = tf.keras.Model(latent_inputs, decoder_outputs, name="decoder")

    return encoder, decoder

def lstm_preprocess(latent_z, seq_length):
    reshape_num = latent_z.shape[0] - seq_length + 1
    slide_z = sliding_window_view(latent_z, (seq_length, latent_z.shape[1]))
    slide_z = slide_z.reshape(reshape_num, seq_length, latent_z.shape[1])
    slide_z = slide_z[:-1]

    dataY = []
    for i in range(0, len(latent_z) - seq_length):
        _y = latent_z[i + seq_length]  # 다음 나타날 z(정답)
        dataY.append(_y)  # dataY 리스트에 추가
    dataY = np.array(dataY)
    
    return slide_z, dataY
    
    
def lstm_model(seq_length, latent_dim, lstm_z):
    
    inputs = tf.keras.Input(shape=(seq_length, latent_dim))
    h = layers.LSTM(32, activation="relu", name='lstm1')(inputs)
    lstm_outputs = layers.Dense(latent_dim, activation='sigmoid')(h)
    lstm = tf.keras.Model(inputs, lstm_outputs, name='lstm')
    lstm.compile(loss='mse', optimizer=tf.keras.optimizers.Adam())
    lstm.fit(lstm_z, dataY, epochs=5, batch_size=256)
    lstm_results = lstm.predict(lstm_z)

    return lstm_results
    
    
if __name__ == '__main__':
    os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
    config = tf.compat.v1.ConfigProto()
    config.gpu_options.allow_growth = True
    session = tf.compat.v1.Session(config=config)

    #---------------------------------사용자 설정 변수---------------------------------------
    FILENAME = "BTC_USDT.csv"
    Time_window = 8
    latent_dim = 2
    #-------------------------------------------------------------------------------------

    #데이터 로딩
    x_train, x_test= data_load_processing(FILENAME, Time_window=Time_window)


In [ ]:
#VAE_model
latent_dim = 2
input_shape = (1, 8, 54)
kernel_size = (2,4)
strides = (2,2)
padding = "valid"
encoder, decoder = vae_model(input_shape, latent_dim, kernel_size, strides, padding)

#VAE
vae = V.VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.fit(x_train, epochs=1, batch_size=128)

z_mean_, z_log_var_, latent_z = vae.encoder.predict(x_train)
#------------------------------------------------------------------------------------------
seq_length = 8

lstm_z, dataY = lstm_preprocess(latent_z, seq_length)

lstm_results = lstm_model(seq_length, latent_dim, lstm_z)

decoder_output = vae.decoder.predict(lstm_results)

In [3]:
#VAE_model
latent_dim = 2
input_shape = (1, 8, 54)
kernel_size = (2,4)
strides = (2,2)
padding = "valid"
encoder, decoder = vae_model(input_shape, latent_dim, kernel_size, strides, padding)
encoder.summary()

#VAE
vae = V.VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.fit(x_train, epochs=10, batch_size=128)

z_mean_, z_log_var_, latent_z = vae.encoder.predict(x_train)

decoded_latent_z = vae.decoder.predict(latent_z)

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(x_train[0][0][0], decoded_latent_z[0][0][0])

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 1, 8, 54)]   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 4, 26)    288         input_1[0][0]                    
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 2, 12)    16448       conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 128, 1, 5)    65664       conv2[0][0]                      
____________________________________________________________________________________________

41.232808150884495

In [4]:
x_train[0][0][0]

array([2.92028172e-01, 2.90334511e-01, 2.94325406e-01, 2.91690876e-01,
       1.28034781e-02, 2.91970143e-01, 1.70846712e-02, 2.91319238e-01,
       2.01296969e-02, 4.47492299e-01, 2.36533311e-01, 3.66674647e-01,
       3.92146587e-01, 2.91061150e-01, 2.12365833e-02, 2.91134819e-01,
       2.79011400e-02, 3.60360194e-01, 1.27683809e-01, 3.65558665e-01,
       2.11723599e-01, 2.91124649e-01, 3.42660671e-02, 2.90319558e-01,
       2.98881262e-02, 3.48278646e-01, 6.07620474e-02, 3.35166460e-01,
       1.17714478e-01, 2.85898890e-01, 2.37780000e-02, 2.85868523e-01,
       3.21074999e-02, 2.99450752e-01, 5.12985089e-02, 2.97449067e-01,
       6.24664374e-02, 2.82898658e-01, 5.55299442e-02, 2.84263507e-01,
       4.31056906e-02, 3.02565338e-01, 2.62291760e-02, 3.02324704e-01,
       4.48243776e-02, 5.33293333e-01, 3.61499602e-02, 9.99928230e-01,
       5.65686608e-01, 4.23772617e-04, 4.13801998e-01, 2.49842430e-01,
       9.89829098e-01, 3.75898567e-02])

In [5]:
decoded_latent_z[0][0][0]

array([0.42252865, 0.42397496, 0.42110223, 0.42557004, 0.01773322,
       0.42546418, 0.03094375, 0.42547044, 0.03870365, 0.46168536,
       0.20816006, 0.3846274 , 0.32000116, 0.4220867 , 0.04146543,
       0.4226823 , 0.05368857, 0.37973073, 0.11159979, 0.38897815,
       0.18066993, 0.42045805, 0.05761619, 0.42210943, 0.05902191,
       0.38305503, 0.06381229, 0.35892898, 0.10345648, 0.41967455,
       0.06633595, 0.41975012, 0.07173923, 0.30713138, 0.03691732,
       0.30958727, 0.05338758, 0.4176122 , 0.0928539 , 0.41606763,
       0.0939569 , 0.30492225, 0.02844059, 0.30822995, 0.03813153,
       0.52328897, 0.01845735, 0.97716916, 0.56466955, 0.        ,
       0.45706686, 0.5068599 , 0.4916169 , 0.04914206], dtype=float32)

In [6]:
#VAE_model
latent_dim = 3
input_shape = (1, 8, 54)
kernel_size = (2,4)
strides = (2,2)
padding = "valid"
encoder, decoder = vae_model(input_shape, latent_dim, kernel_size, strides, padding)
encoder.summary()

#VAE
vae = V.VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.fit(x_train, epochs=10, batch_size=128)

z_mean_, z_log_var_, latent_z = vae.encoder.predict(x_train)

decoded_latent_z = vae.decoder.predict(latent_z)

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(x_train[0][0][0], decoded_latent_z[0][0][0])

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 1, 8, 54)]   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 4, 26)    288         input_3[0][0]                    
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 2, 12)    16448       conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 128, 1, 5)    65664       conv2[0][0]                      
____________________________________________________________________________________________

42.59124723988586

In [7]:
#VAE_model
latent_dim = 4
input_shape = (1, 8, 54)
kernel_size = (2,4)
strides = (2,2)
padding = "valid"
encoder, decoder = vae_model(input_shape, latent_dim, kernel_size, strides, padding)
encoder.summary()

#VAE
vae = V.VAE(encoder, decoder)
vae.compile(optimizer=tf.keras.optimizers.Adam())
vae.fit(x_train, epochs=10, batch_size=128)

z_mean_, z_log_var_, latent_z = vae.encoder.predict(x_train)

decoded_latent_z = vae.decoder.predict(latent_z)

def MAPE(y_test, y_pred):
    return np.mean(np.abs((y_test - y_pred) / y_test)) * 100 
    
MAPE(x_train[0][0][0], decoded_latent_z[0][0][0])

Model: "encoder"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 1, 8, 54)]   0                                            
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, 32, 4, 26)    288         input_5[0][0]                    
__________________________________________________________________________________________________
conv2 (Conv2D)                  (None, 64, 2, 12)    16448       conv1[0][0]                      
__________________________________________________________________________________________________
conv3 (Conv2D)                  (None, 128, 1, 5)    65664       conv2[0][0]                      
____________________________________________________________________________________________

108.89255136453406

## latent dim 은 크게 영향을 끼치지 않는것 같다